In [1]:
# coding : 'utf-8'
# coding:unicode_escape
import pandas as pd
usecols1 = ['订单号','SKU','序号']
usecols2 = ['平台订单号','SKU','数量','补寄数量','最早时间']
origin = pd.read_excel('D:/work/after_sale/总售后2022-04-15.xlsx')
origin.head()

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,操作代码,Refund,备注
0,1,2022-01-02,2022-03-22,赵婧,维禄,111-7779749-5070667,USAN1027671-5,1,已校对,少件(包装未破损)PartH,"国内补寄配件PartH,退部分款",3-5-0$H&1-2-0,222.77,"少8个partH，已登记已发出,1/10退款。"
1,2,2022-01-01,2022-01-02,赵婧,维禄,114-7520796-7137863,USAN1027671-5,1,已校对,无理由退货(不想要),"退部分款,买家使用我司Return label退货",1-2-0&2-6-0,222.88,拒收失败退回退款
2,3,2022-01-02,2022-01-02,陈丽君,信盒,113-1566999-8610619,USAN1023799-4,1,解决中,询问物流(改地址),已解决,5-4-0,0.00,分包，两箱到一箱2.8查到另一箱也已妥投
3,4,2022-01-02,2022-01-02,陈丽君,信盒,113-1566999-8610619,USAN1023799-3,2,解决中,询问物流(改地址),已解决,5-4-0,0.00,分包，两箱到一箱2.8查到另一箱也已妥投
4,5,2022-01-01,2022-01-02,郄思颖,宫本,113-5369606-1680207,USAN1029122-3,1,已校对,无理由退货(不想要),"退部分款,买家自行退货",1-2-0&2-5-0,254.98,不想要了（扣80运费）


In [5]:
import csv
import os
import pymysql
from pathlib import Path
from openpyxl import Workbook
import pandas as pd
import re
import numpy as np
from database_read import read
# def read(basename, tablename):
#     connc = pymysql.Connect(
#         host='127.0.0.1',
#         user='root',
#         password='123456',
#         database='%s' % basename,
#         port=3306,
#         charset='utf8'
#     )

#     cur = connc.cursor()
#     sql = 'select * from `%s`;' % tablename
#     cur.execute(sql)

#     des = cur.description
#     title = []
#     for j in range(len(des)):
#         title.append(des[j][0])

#     origin = cur.fetchall()
#     origin = pd.DataFrame(list(origin))
#     origin.columns = title
#     return origin




In [7]:
inventory = read('inventory','自发货源数据new1')

In [8]:
inventory["补寄数量"] =inventory["补寄数量"].fillna(0)
inventory_buji = inventory
inventory_buji.head()

KeyError: '补寄数量'

In [ ]:
inventory[inventory['补寄数量']>0]

In [ ]:
origin_buji = origin[origin["客服操作"].str.contains('海外仓补寄')]
origin_buji = origin_buji.reset_index(drop = True)
origin_buji['补寄数量'] = 0
origin_buji.head()

In [ ]:
def find_all(string, sub):
    start = 0
    pos = []
    while True:
        start = string.find(sub, start)
        if start == -1:
            return pos
        pos.append(start)
        start += len(sub)


# print(find_all('3-2-0$HOG101431-02num1&3-1-0$HOG101431-02num1', 'num'))
print(find_all(str(origin_buji.loc[2,'操作代码']), 'num'))

In [ ]:
inventory_in_buji = pd.DataFrame(columns=['id', '登记日期', '更新日期', '登记人', '店铺', '订单号', 'SKU', '序号', '订单状态', '顾客反馈', '客服操作','操作代码','Refund', '备注'])
dengji_buji = pd.DataFrame(columns=['id', '登记日期', '更新日期', '登记人', '店铺', '订单号', 'SKU', '序号', '订单状态', '顾客反馈', '客服操作','操作代码', 'Refund', '备注','补寄数量','数据库补寄数量'])


In [45]:
code = ['3-1-0','3-2-0','3-3-0','3-4-0']
import time
for i in range(len(origin_buji)):
    code_split_list = origin_buji.loc[i,'操作代码'].split('&')
#     code_split_list = '3-1-0$ESAN1024542num1&2-4-0'.split('&')
    for j in code :
        num_sum = 0
        for k in range(len(code_split_list)):
            
            if j in  code_split_list[k]:
#                 code_sit = code_split_list[k]
                if "num" not in code_split_list[k] and "ship" not in code_split_list[k]:
#                      origin_buji.loc[i,'补寄数量'] = 1
                     num_sum  = num_sum+1
#                 if len(find_all(code_split_list[k],'num')) == 2:
#                     num_n = 0
#                     num = find_all(code_split_list[k],'num')        
#                     for k in num:
#                         num_n += int(code_split_list[k][k+3:k+4])
#                     origin_buji.loc[i,'补寄数量'] = num_n
    #             print(origin_buji.loc[i,'操作代码'],num_n,'***************************')
                if len(find_all(code_split_list[k],'num')) == 1:
                    num = code_split_list[k].index('num')
                    num_n1 = int(code_split_list[k][num+3:num+4])
                    num_sum =  num_n1
#                     origin_buji.loc[i,'补寄数量'] = num_n1
                if 'ship' in code_split_list[k]  : 
                    num_ship = code_split_list[k].index('ship')
                    ship_n = int(code_split_list[k][num_ship+4:num_ship+5])
                    num_sum = num_sum + ship_n
#                     origin_buji.loc[i,'补寄数量'] = ship_n
        origin_buji.loc[i,'补寄数量'] = origin_buji.loc[i,'补寄数量']+num_sum
    
 
    print(origin_buji.loc[i,'操作代码'],origin_buji.loc[i,'补寄数量'])

now = time.strftime("%Y-%m-%d-%H_%M",time.localtime(time.time())) 
origin_buji.to_excel(excel_writer="E:/origin_buji/"+now+r"origin_buji.xlsx", index=False,encoding='utf-8')

3-1-0 1
3-1-0 1
3-1-0$USAN1030934-3num1 1
1-2-0&2-6-0ship2&3-1-0$USAN1024789-2num1 1
3-1-0 1
3-2-0 1
3-2-0$USAN1026301-2num1 1
3-1-0$USAN1026771-5num1 1
3-1-0 1
3-1-0$ESAN1024542num1&2-4-0 1
3-2-0&1-2-0 1
3-1-0 1
3-1-0$USAN1018799-6num1 1
3-1-0 1
3-4-0$L 1
3-2-0$USAN1022780-6num1 1
3-1-0 1
3-5-0$Fship1&3-2-0$USAN1026301-1num1 1
3-2-0&1-2-0 1
3-1-0 1
3-1-0 1
3-1-0 1
3-2-0&1-2-0 1
3-2-0 1
3-2-0&1-2-0 1
3-2-0$HOG101439-02num1 1
3-2-0 1
3-1-0 1
3-4-0 1
3-2-0&1-2-0 1
3-2-0 1
1-2-0&3-2-0$HOG101439-02num2 2
3-1-0$USAN1017139-2num1 1
3-2-0&1-2-0 1
3-1-0$USAN1023786-2num1 1
3-4-0$J 1
3-1-0 1
1-2-0&3-2-0$USAN1022780-8num1 1
3-1-0 1
3-1-0 1
3-2-0&1-2-0 1
3-2-0$USAN1022780-4num1 1
3-2-0&1-2-0 1
3-2-0$USAN1022780-8num1 1
3-2-0&1-2-0 1
3-4-0$J 1
3-2-0 1
3-4-0 1
3-2-0 1
3-2-0&1-2-0 1
1-2-0&3-1-0$USAN1034651-3num1 1
3-1-0&2-4-0 1
1-2-0&2-5-0&3-2-0$USAN1023809-4num1 1
3-2-0 1
3-1-0&2-5-0 1
3-2-0&1-2-0 1
3-1-0 1
3-2-0 1
3-2-0&1-2-0 1
3-1-0 1
3-4-0$J 1
3-1-0&2-4-0 1
3-2-0&1-2-0 1
3-1-0$USAN1023809-3num1 

3-1-0 1
3-1-0$USAN1030934-2num1 1
3-2-0 1
3-1-0 1
3-1-0$USAN1031524-2num1 1
3-4-0&1-2-0 1
3-4-0 1
3-1-0$USAN1030934-2num1 1
2-6-0&3-4-0$A 1
3-2-0 1
3-2-0 1
3-2-0 1
3-1-0 1
3-1-0 1
1-2-0&3-4-0&2-6-0 1
3-2-0 1
3-1-0$USAN1034651-3num1&1-1-0 1
3-4-0$G 1
3-1-0 1
3-5-0$H#F1#I#J#L99ship1&3-4-0$U99ship1 1
3-1-0$USAN1035086-5num1 1
3-4-0&1-2-0 1
3-1-0 1
3-2-0&1-2-0 1
3-1-0 1
1-2-0&3-1-0 1
3-4-0 1
3-1-0 1
3-1-0$USAN1029512-4num1&2-4-0 1
3-2-0$USAN1022780-4num1 1
3-2-0 1
3-2-0$USAN1022780-3num1 1
3-1-0 1
3-2-0$HOG101431-03num1&1-2-0 1
3-1-0$USAN1031011-2num1 1
3-2-0&1-2-0 1
3-1-0 1
3-1-0 1
1-2-0&3-2-0$USAN1018795-5num1 1
3-1-0 1
3-2-0 1
3-2-0 1
3-2-0 1
3-1-0$USAN1030934-2num1 1
3-1-0 1
3-2-0$USAN1025238-1num1 1
3-1-0$USAN1023384-5num1 1
3-1-0 1
3-1-0 1
3-2-0 1
1-2-0&3-1-0&2-3-0 1
3-2-0 1
3-2-0&1-2-0 1
3-1-0 1
1-2-0&2-4-0&3-2-0$USAN1027459-2num1 1
3-2-0$USAN1017139-5num1 1
3-1-0 1
3-1-0$USAN1024785-2num1 1
3-4-0 1
3-4-0 1
3-1-0 1
1-2-0&2-6-0ship1&3-1-0$USAN1018795-3num1 1
3-2-0$USAN1017139-6num1&1

3-2-0$USAN1027459-2num1 1
3-2-0 1
3-1-0$ESAN1024541num1 1
3-1-0 1
3-1-0$USAN1024791-1num1&1-1-0 1
3-1-0 1
3-1-0$USAN1030934-5num1 1
1-1-0&3-1-0$HOG101439-02num1&3-5-0 1
3-2-0&2-3-0 1
3-1-0$HOG100865-01num1 1
3-1-0$HOG100865-01num1 1
3-1-0 1
3-2-0 1
3-1-0 1
3-1-0 1
3-1-0 1
3-2-0 1
3-2-0 1
3-2-0 1
2-9-0&3-1-0$USAN1034749-1num1 1
3-2-0$USAN1027460-4num1&1-2-0 1
3-2-0$USAN1026261-1num1 1
3-2-0$USAN1023811-4num1&1-2-0 1
3-1-0$USAN1023426-1num1 1
3-3-0$USAN1011322-6num1 1
3-2-0$USAN1029591-1num1 1
3-1-0$USAN1018441-2num1 1
3-1-0$USAN1023938-2num1 1
3-1-0$USAN1034749-2num1 1
3-4-0$J 1
3-2-0$HOG101431-03num1 1
3-2-0$USAN1018797-3num1 1
3-1-0$USAN1025123-3num1 1
3-2-0$USAN1029512-4num1&3-1-0$USAN1029512-4num1 2
3-2-0$USAN1029512-2num1 1
3-1-0$USAN1023801-2num1 1
3-1-0$USAN1034510-5num1 1
3-1-0&2-4-0 1
3-1-0$USAN1020595-1num1 1
3-2-0$USAN1025838-2num1 1
3-2-0$USAN1023767-3num1&2-6-0ship1 1
3-1-0$PTG103482-01num1 1
3-1-0$USAN1034749-1num1 1
3-2-0$USAN1023767-2num1&1-2-0 1
3-1-0$HOG100865-02num1 1

3-1-0$USAN1035086-5num1 1
3-1-0$USAN1011336-2num1 1
3-1-0$USAN1031424-3num1&1-1-0 1
3-3-0$USAN1023384-2num1 1
3-1-0$USAN1023801-3num1 1
3-1-0$USAN1034654-3num1&2-6-0ship1 1
3-1-0$USAN1034691-2num1 1
3-1-0$USAN1034691-2num1 1
3-1-0$USAN1023812-2num1 1
3-1-0$USAN1023811-5num1 1
3-2-0$USAN1024785-2num1 1
3-1-0$USAN1032096-4num1&1-1-0 1
3-2-0$USAN1011338-1num1 1
3-1-0$USAN1031424-5num1 1
3-1-0$USAN1029592-1num1&2-4-0 1
3-4-0$J 1
3-3-0$USAN1023807-1num1 1
3-1-0$USAN1032096-11num1 1
3-3-0$USAN1034650-13num1 1
3-1-0$USAN1034821-2num1 1
3-1-0$USAN1035090-3num1 1
3-1-0$USAN1029449-2num1 1
3-1-0$USAN1030935-6num1 1
3-1-0$USAN1023944-2num1 1
3-1-0$USAN1032096-11num1&1-1-0&2-3-0ship1 1
3-1-0$USAN1034749-2num1 1
3-1-0$USAN1022497-3num1 1
3-1-0$USAN1030092-2num1 1
5-1-0&3-1-0$USAN1034510-5num1 1
3-2-0$HOG101431-03num1 1
3-2-0$USAN1018606-3num1 1
3-2-0$USAN1034818-3num1 1
3-2-0$USAN1027459-3num1 1
3-3-0$USAN1031424-1num1 1
3-2-0$USAN1011315-3num1 1
3-1-0$USAN1029512-3num1 1
3-3-0$USAN1031424-2num1 1


3-2-0$USAN1025837-2num1 1
3-1-0$USAN1020595-1num1 1
3-1-0$USAN1023944-2num1 1
3-1-0$USAN1032096-5num1 1
2-9-0&3-1-0$USAN1025124-1num1 1
3-2-0$USAN1023937-4num1 1
3-1-0$USAN1031524-2num1 1
3-1-0$USAN1025124-2num1 1
3-2-0$USAN1023944-5num1 1
3-2-0$USAN1018797-1num1 1
3-4-0$L99 1
3-2-0$USAN1023938-5num1 1
3-1-0$USAN1027460-5num1 1
3-3-0$USAN1011322-3num2 2
3-2-0$USAN1025238-3num1 1
3-1-0$USAN1023801-1num1 1
3-1-0$USAN1018795-5num1 1
3-1-0$USAN1011338-1num1 1
3-2-0$USAN1034654-3num1 1
3-4-0$L99 1
3-1-0$USAN1026771-4num1 1
3-3-0$USAN1023809-4num1&2-7-0ship1&1-1-0 1
3-1-0$USAN1025124-2num1 1
3-1-0$USAN1025123-2num1 1
3-1-0$USAN1025123-2num1 1
3-4-0$G 1
3-1-0$USAN1023788-6num1 1
3-1-0$HOG100865-02num1 1
3-3-0$USAN1029451-3num1 1
3-2-0$USAN1031424-5num1 1
3-3-0$USAN1023801-1num1 1
3-1-0$HOG100865-03num1 1
3-1-0$USAN1023811-3num1 1
3-4-0$L99&1-2-0 1
3-1-0$USAN1034653-9num1 1
3-1-0$USAN1026771-6num1 1
3-2-0$USAN1031524-2num1 1
3-1-0$USAN1023811-3num1 1
3-1-0$USAN1023811-1num1&2-4-0 1
3-1-0$USAN1

3-1-0$USAN1025123-2num1 1
3-1-0$USAN1030092-2num1 1
3-1-0$USAN1018606-6num1 1
3-3-0$USAN1034650-19num1 1
3-1-0$USAN1023938-4num1 1
3-1-0$USAN1026261-1num1 1
3-1-0$USAN1023811-6num1 1
3-1-0$USAN1026771-1num1 1
3-1-0$USAN1026771-2num1 1
3-1-0$USAN1023809-1num1 1
3-1-0$USAN1018798-1num1 1
3-1-0$USAN1034510-5num1 1
2-6-0&5-1-0&3-1-0$USAN1027671-1num1 1
3-1-0$USAN1022497-6num1 1
3-1-0$USAN1029592-1num1 1
3-1-0$USAN1029512-3num1 1
3-2-0$USAN1011322-6num1 1
3-2-0$USAN1023938-2num1 1
3-1-0$USAN1023938-5num1 1
3-2-0$USAN1026771-5num1 1
3-1-0$USAN1031424-5num1 1
3-1-0$USAN1030934-3num1 1
3-1-0$USAN1026261-1num1 1
3-2-0$USAN1029451-1num1 1
3-1-0$USAN1024785-2num1 1
3-2-0$USAN1024789-1num1 1
3-1-0$USAN1023767-2num1 1
3-1-0$PTG103482-01num1 1
3-2-0$USAN1023799-1num1 1
3-1-0$USAN1030935-9num1 1
3-1-0$USAN1035676-5num1 1
3-1-0$USAN1034650-24num1 1
3-1-0$USAN1029931-4num1 1
3-1-0$USAN1011322-2num1 1
3-2-0$USAN1018505-5num1 1
3-1-0$USAN1022706-5num1 1
3-1-0$USAN1029204-1num1 1
3-1-0$USAN1026261-2num2 2

3-3-0$USAN1022497-6num1 1
3-1-0$USAN1034817-3num1 1
3-1-0$USAN1022497-1num1 1
3-1-0$USAN1022497-1num1 1
3-1-0$USAN1029932-3num1 1
3-1-0$USAN1016483-6num1 1
3-1-0$USAN1025123-3num1 1
3-1-0$USAN1035028-2num1 1
3-1-0$USAN1016483-6num1 1
3-1-0$USAN1034821-2num1 1
3-1-0$USAN1013770-1num1 1
3-1-0$USAN1023937-6num1 1
3-1-0$USAN1017139-1num1 1
3-1-0$USAN1031524-2num1 1
3-2-0$USAN1022653-2num1 1
3-1-0$USAN1023541-2num1&2-7-0ship1 1
3-2-0$USAN1026771-2num1 1
3-1-0$USAN1026771-1num1&1-2-0 1
3-2-0$USAN1018441-2num1 1
3-2-0$USAN1027459-2num1 1
3-1-0$USAN1032096-1num1 1
3-2-0$USAN1026259-1num1 1
3-1-0$USAN1030092-2num1 1
3-1-0$USAN1026771-3num1 1
1-2-0&3-1-0$USAN1031424-5num1 1


In [46]:
origin_buji_table = origin_buji.pivot_table(index = '订单号',aggfunc = { '补寄数量':'sum'} )
origin_buji_table = origin_buji_table.reset_index()
origin_buji_table
# origin_buji_table.iloc[:,0]

,订单号,补寄数量
0,01-08167-85210,1
1,01-08177-31089,1
2,01-08286-28607,1
3,01-08369-20020,1
4,03-08190-35877,1
...,...,...
3046,NF10834863422,1
3047,NF10834863456,1
3048,NF10834863534,1
3049,NF10834863600,1


In [47]:
import time
for i in range(len(origin_buji)):
#     print(i)
    
    if (origin_buji.iloc[i,5] in inventory_buji['平台订单号'].tolist()):
        sku_sit = inventory_buji['平台订单号'].tolist().index(origin_buji.iloc[i,5])
        table_sit = origin_buji_table['订单号'].tolist().index(origin_buji.iloc[i,5])
#         print(origin_buji.iloc[i,6],' ' ,inventory_buji.iloc[sku_sit,2])
#         if (origin_buji.iloc[i,6] == inventory_buji.iloc[sku_sit,2]) and (origin_buji.iloc[i,14] != inventory_buji.iloc[sku_sit,4]) :
        if int(origin_buji_table.iloc[table_sit,1]) != int(inventory_buji.iloc[sku_sit,3]) :

            print(origin_buji.loc[i,'订单号'],'***********')
#             print(origin_buji[origin_buji['订单号']==origin_buji.iloc[i,5]])
#             print('*',origin_buji.iloc[i,14] ,' ', inventory_buji.iloc[sku_sit,4],'*')
            origin_output1 = pd.DataFrame(origin[origin['订单号']==origin_buji.iloc[i,5]])
            origin_output1['补寄数量'] = origin_buji_table.iloc[table_sit,1]
            origin_output1['数据库补寄数量'] = inventory_buji.iloc[sku_sit,3]
            
            dengji_buji=dengji_buji.append(origin_output1)
     
            
# #             print("wu")
    else:
        print(origin_buji.iloc[i,5],'-------')
        table_sit = origin_buji_table['订单号'].tolist().index(origin_buji.iloc[i,5])
        origin_output = pd.DataFrame(origin[origin['订单号']==origin_buji.iloc[i,5]])
        origin_output['补寄数量'] = origin_buji_table.iloc[table_sit,1]
            
        inventory_in_buji=inventory_in_buji.append(origin_output)


#加个去重！！！！！

# fname="D:/logs/"+now+r"report.csv"   
# dengji_buji.head()
# dengji_buji.to_excel(excel_writer="E:/登记补寄数量/登记_补寄数量不符.xlsx", index=False,encoding='utf-8')
# inventory_in_buji.to_excel(excel_writer="E:/错误订单/不在_inventory的订单以及售后补寄数.xlsx", index=False,encoding='utf-8')

114-5002307-5673839 ***********
111-0944292-7677015 ***********
111-8846098-9299450 ***********
2812136020033 ***********
2812136020033 ***********
114-1549502-5444226 ***********
113-5950532-2061022 ***********
112-0292504-5927460 ***********
4863097350749 ***********
4863097350749 ***********
111-1442423-6921057 ***********
111-3045217-0274639 ***********
7863637637147 ***********
5862931342003 ***********
2863453443251 ***********
111-6771387-2289028 ***********
114-4465998-0971444 ***********
111-4170723-4466659 ***********
111-4170723-4466659 ***********
7870029103730 ***********
111-2801614-9058637 ***********
113-5139423-3346616 ***********
114-9295386-3894610 ***********
113-9886241-7851405 ***********
111-6001787-0724266 ***********
113-8654912-2002619 ***********
2870137597294 ***********
114-7331978-4190654 ***********
112-5719620-9678657 ***********
113-5017010-9021852 ***********
8863375973419 ***********
113-5373287-3007456 ***********
113-7636278-5941846 ***********
8870

CS377626057 ***********
CS390897964 ***********
114-8377452-8321856 ***********
CS350546343 ***********
112-9510015-5407408 ***********
8870829453074 ***********
CS370802567 ***********
114-0551704-1716204 ***********
7870789417654 ***********
7870776804617 ***********
114-9499231-1387468 ***********
114-6652261-1256205 ***********
114-6652261-1256205 ***********
111-1468986-9829015 ***********
113-3668353-5724262 ***********
113-8403787-3101868 ***********
114-8602496-6793862 ***********
114-7112793-2862647 ***********
111-7153374-4197838 ***********
113-2059658-6081042 ***********
26-08406-02489 ***********
113-7511352-1292201 ***********
113-9208570-1289864 ***********
3870607862617 ***********
25-08396-50463 ***********
113-1570315-5005045 ***********
111-4981274-0897040 ***********
114-5578054-8759417 ***********
2870569001283 ***********
113-4589398-2786655 ***********
15-08399-34165 ***********
6870713528074 ***********
NF10834861588 ***********
CS366340471 ***********
CS3835462

10-08442-08025 ***********
15-08465-37692 -------
CS386754963 ***********
CS394264455 ***********
113-2391693-9254663 -------
CS395995863 -------
CS395984222 -------
113-2856636-0140232 ***********
9870879837708 ***********
NF10834863534 -------
113-4589274-1616207 -------
NF10834863642 -------
CS394843219 -------
113-9213863-5993851 -------
3870970137959 -------
CS395878829 -------
111-2949833-5130623 -------
111-3871897-1955454 -------
113-1580138-6341027 ***********
1870894523863 -------
114-0290908-0957011 ***********
113-9277237-5335466 ***********
CA393831362 ***********
114-8256542-9546606 ***********
14-1513473-7053052 -------
111-6665206-0800245 ***********
113-9249337-0121867 ***********
114-8252925-6100208 -------
111-6260868-3222630 -------
111-8521741-5809060 -------
111-8521741-5809060 -------
114-1036936-6167467 -------
112-2045043-6951405 ***********
2870968519956 -------
2870968519956 -------
112-4616617-4040248 ***********
111-6049144-2317021 ***********
113-2153032-1

In [48]:
now = time.strftime("%Y-%m-%d-%H_%M",time.localtime(time.time())) 

# dengji_buji = dengji_buji.drop_duplicates(keep = False)

dengji_buji.to_excel(excel_writer="E:/登记补寄数量/"+now+r"登记_补寄数量不符.xlsx", index=False,encoding='utf-8')



In [49]:
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 
# inventory_in_buji = inventory_in_buji.drop_duplicates(keep = 'first')
inventory_in_buji.to_excel(excel_writer="E:/错误订单/"+now+r"不在_inventory的订单以及售后补寄数.xlsx", index=False,encoding='utf-8')